# Step 1: Data Importing 


In [ ]:
#Importing Librariees
import pandas as pd 
import seaborn as sns 
import numpy as np 
import matplotlib.pyplot as plt 

In [ ]:
#Reading File
google_play_df= pd.read_csv("data/googleplaystore.csv")  
google_play_reviews_df = pd.read_csv("data/googleplaystore_user_reviews.csv")

# Step 2: Data Cleaning




In [ ]:
#Shifting Values for row 10472
google_play_df.loc[10472] = google_play_df.loc[10472][1:].shift(periods=1, fill_value ='LIFESTYLE')
google_play_df.loc[10472]['App'] = 'Life Made Wi-Fi Touchscreen Photo Frame'
google_play_df.loc[10472]

In [ ]:
#Replacing NAN Values
google_play_df['Rating'] = google_play_df['Rating'].astype(float)  
google_play_df.at[9148, "Type"] = "Free" 
google_play_df.at[10472,'Genres'] = 'Lifestyle'
google_play_df.at[4490,"Android Ver"]= "8.0 and up" 
google_play_df.at[4453, "Android Ver"]="8.0 and up"
google_play_df.at[15,"Current Ver"] = "1.0.0"
google_play_df.at[1553,"Current Ver"] = "1.0.0" 
google_play_df.at[6322,"Current Ver"] = "1.0.0"
google_play_df.at[6803,"Current Ver"] = "1.0.0"
google_play_df.at[7333,"Current Ver"] = "1.0.0"
google_play_df.at[7407,"Current Ver"] = "1.0.0" 
google_play_df.at[7730, "Current Ver"] = "1.0.0"
google_play_df.at[10342,"Current Ver"] = "1.0.0"

In [ ]:
#function will convert a value from a string for size into a float in megabytes.
def size_in_megabyte_converter(val):
    if val.endswith('M'):
        val = float(val.replace('M',''))
    elif val.endswith('k'):
        val = float(val.replace('k',''))/1000
    else:
        val = float('NaN')
    return val

#Function will reassign a genre value as what comes first after a semicolon.
def genres_splitter(genre):
    if ';' in genre:
        genre = genre.split(';')[1]
    else:
        genre= 'General '+genre
    return genre

#Converts price column into a float by dropping the money sign.
def price_to_float_converter(value):
    if value.startswith('$'):
        value = float(value.replace('$',''))
    else:
        value = float(value)
    return value

In [ ]:
#Applying the functions to the columns in our dataset
google_play_df['Genres'] = google_play_df['Genres'].apply(lambda x: genres_splitter(x))
google_play_df['Price'] = google_play_df['Price'].apply(lambda x: price_to_float_converter(x))
google_play_df['Size'] = google_play_df['Size'].apply(lambda x: size_in_megabyte_converter(x))

In [ ]:
#Dropping Duplicates
to_drop = google_play_df[google_play_df.duplicated(keep='first')].sort_values(by="App").index

google_play_df.drop(to_drop,inplace=True)

In [ ]:
#Unadjusted Data
unadjusted_rating = sns.distplot(google_play_df['Rating']) 
to_agg_unajusted = sns.distplot(google_play_df['Rating'])
print(to_agg_unajusted.agg(["mean", "median", "std"]))

In [ ]:
#Nan Values Dropped
all_nan_dropped_ratings = sns.distplot(google_play_df['Rating'].dropna())
to_agg_dropped = sns.distplot(google_play_df['Rating'].dropna())
print(to_agg_dropped.agg(["mean", "median", "std"]))

In [ ]:
#Mean Applied
google_play_df["Rating"].mean()
mean_ratings_applied = sns.distplot(google_play_df['Rating'].fillna(value=google_play_df['Rating'].mean()))
to_agg_mean = google_play_df['Rating'].fillna(value=google_play_df['Rating'].mean()) 
print(to_agg_mean.agg(["mean", "median", "std"])) 

In [ ]:
#Median Applied
google_play_df["Rating"].median() 
median_ratings_applied = sns.distplot(google_play_df['Rating'].fillna(value=google_play_df["Rating"].median())) 
to_agg_median = google_play_df['Rating'].fillna(value=google_play_df['Rating'].median()) 
print(to_agg_median.agg(["mean", "median", "std"]))

In [ ]:
#Nan Value Count 
google_play_df.isna().sum()

# Question 1: 


In [ ]:
#Grouping Data
categorical_grouping = google_play_df.groupby(["Category","Rating"]).head() 
categorical_grouping

In [ ]:
#Rating Mean Sorted 
rating_mean_sorted = categorical_grouping.groupby(["Category"]).mean() 
rating_mean_sorted

In [ ]:
#Ordered Categories
cat_rating_series = google_play_df.groupby("Category").agg(np.mean)["Rating"].sort_values(ascending=False)
cat_rating_series 

In [ ]:
#Graphed Values
plt.figure(figsize=(20,5))
q1_visual = sns.barplot(x="Category", y="Rating", data=categorical_grouping)
q1_visual.set_xticklabels(q1_visual.get_xticklabels(), rotation=90) 
q1_visual.set_title("Rating By Category")  

In [ ]:
#Top 5 Category Means 
top_5 = rating_mean_sorted.loc[rating_mean_sorted["Rating"] >= 4.26]  
top_5_ordered = top_5.sort_values(['Rating'])
top_5_ordered

In [ ]:
#Agg Functions for Grouped Dataframe 
df= pd.read_csv("data/googleplaystore.csv") 
grouped = df.groupby(["Category", "App"])["Rating"].agg(['mean', 'min', 'max', 'std'])        
grouped

# Question 2:


In [ ]:
q2_subset = google_play_df[(google_play_df['Rating'].isna() ==False)\
                           & (google_play_df['Size'].isna() == False)]
q2_smallset = q2_subset[q2_subset['Size'] <1.0]
q2_sm_g = sns.jointplot(data = q2_smallset, x='Size',y='Rating', kind='hex')


In [ ]:
q2_midset = q2_subset[(q2_subset['Size']>=1.0)&(q2_subset['Size']<10.0)]
q2_m_g = sns.jointplot(data=q2_midset, x='Size', y='Rating',kind='hex')

In [ ]:
q2_largeset = q2_subset[q2_subset['Size']>=10.0]
q2_lg_g = sns.jointplot(data=q2_largeset, x='Size', y='Rating', kind="hex")

In [ ]:
big_graph = sns.jointplot(data=q2_subset, x='Size', y='Rating', kind="hex",height=8)
big_graph.fig.suptitle('Distribution Density of Size vs. Rating')
plt.show()

What is the Correlation between Size and Rating?


In [ ]:
q2_subset['Size'].corr(q2_subset['Rating'], method='pearson')

It is 0.08233. Very Low almost no correlation

# For this next section, choose a category of Playstore Apps
# Category Chosen: Education

# Question 3:


In [ ]:
#Exploring Unique Genre Enteries
df["Genres"].unique()  
print(len(education)) 
education = google_play_df.loc[(df["Category"] == "EDUCATION")]  
education["Genres"].unique()

In [ ]:
#Setting the index by Genre
education.set_index(["Category", "Genres"], inplace = True)

In [ ]:
#Value Count Of Each Genre 
education = education.groupby('Genres')['App'].nunique()  
education

In [ ]:
#x = ["General Education", "Creativity", "Music & Video", "Action & Adventure", "Pretend Play", "Brain Games"] 
y = [107, 3, 1, 1, 4, 3]

In [ ]:
#Graphed 
genres = sns.barplot(x=x, y=y)
genres.set_xticklabels(genres.get_xticklabels(), rotation=90) 
genres.set_title("Genre Distribution") 


# Question 4: 

What we will investigate: How does sentiment affect the rating, and how do other factors affect the sentiment?

In [ ]:
#Import the data and look at how many unique apps there are
google_play_reviews_df = pd.read_csv("data/googleplaystore_user_reviews.csv")
len(google_play_reviews_df['App'].unique())

In [ ]:
#Group the data by apps and get the mean of the sentiment and polarity
grouped_play_reviews = google_play_reviews_df.groupby(google_play_reviews_df['App']).mean()

#Then we drop any nan's from the sentiment data
grouped_play_reviews.dropna(inplace=True)

In [ ]:
#Combine the google play data with the grouped sentiment data into it's own dataframe
merged_data = pd.merge(left=google_play_df, right=grouped_play_reviews, how='inner',on='App')
print(len(merged_data))


In [ ]:
#We Cahnge the name of columns to make them more in line with already existing columns in our data
merged_data = merged_data.rename(columns= {'Sentiment_Polarity':'Sentiment Polarity', 'Sentiment_Subjectivity':'Sentiment Subjectivity'})
merged_data['Reviews'] = merged_data['Reviews'].astype(int)

In [ ]:
#We create a column called polarity type that tells us where a review was positive or negative
merged_data['Polarity Type'] = merged_data['Sentiment Polarity'].apply(lambda x: 'Positive' if x>0 else 'Negative')

In [ ]:
#We create a facet grid that give us data on how whether our data is affected by type or content rating
g = sns.FacetGrid(merged_data, row= 'Type',col = 'Content Rating', hue= 'Polarity Type')
g = g.map(plt.scatter,'Sentiment Polarity', 'Rating')

In [ ]:
# We create a boxplot that compares the polarities and the rating
sns.boxplot(data= merged_data, x='Polarity Type', y='Rating')

In [ ]:
# We create another boxplot comparing Sentiment Subjectivity to the polarities
sns.boxplot(data= merged_data, x='Polarity Type', y='Sentiment Subjectivity', whis =3)